In [ ]:
import numpy as np
from coverage import *
from datetime import datetime, timezone, timedelta

from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import branca
import folium

! mkdir tmp

In [ ]:
from landsat import Instrument, Platform, Scene

In [ ]:
start_dt = datetime.fromisoformat(Scene.start_utc)
num_days = 8

tles = gen_sats(
    # sat_nos=[Platform.norad_id] # How to best handle multiple platforms? (TLE vs. SPG4 model too)
    sat_nos=[39084,49260]
)

inst = camera_model(
    name=Instrument.name, 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=start_dt.year,
    start_mo=start_dt.month, 
    start_day=start_dt.day, 
    days=num_days, 
    step_min=Instrument.img_period)

xcell_size = ycell_size = .1

In [ ]:
## Batch FOV generation over N satellites - TODO: build multiple sats into config/ main script
gdfs = []
for tle in tles:
    sat = tle[0]
    fov_df = forecast_fovs(sat, times, inst)
    gdfs.append(fov_df)
fov_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")

## Filter shapes crossing anti-meridian - also in main function
## TODO: Switch to stactools solution for this
fov_df["lonspan"] = fov_df.bounds['maxx'] - fov_df.bounds['minx']
fov_df = fov_df[fov_df["lonspan"] < 20].copy()

## Create cmap for unique satellites and create color column
sat_ids = list(fov_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
fov_df['color'] = fov_df['id'].apply(cmap)

## Save to geojson based on sat name
for satname in fov_df.satellite.unique():
    fov_df[fov_df.satellite==satname].to_file("./tmp/{}_fovs.geojson".format(satname.replace(" ", "_")))
fov_df

In [ ]:
## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# aoi =  world[world.name == "Brazil"].geometry
# aoi =  world[world.continent == "South America"].geometry

## Or read in aoi from .geojson
# aoi = world[world.name == "United States of America"].geometry # Includes Alaska...
aoi = gpd.read_file('./aois/eastern_us.geojson').geometry # ...so use AOI for subsection of US

## Filter fov_df by aoi
xmin, ymin, xmax, ymax= aoi.total_bounds
fov_df = fov_df.cx[xmin: xmax, ymin: ymax]

In [ ]:
## Coverage data analysis for single satellite/ batch of satellites
grid, grid_shape = calculate_revisits(fov_df, aoi, grid_x=xcell_size, grid_y=ycell_size)
grid.to_file('./tmp/all_revisits.geojson')
grid.n_visits.fillna(0).describe()

In [ ]:
m = revisit_map(grid, grid_shape, grid_x=xcell_size, grid_y=ycell_size)
m#.save("./tmp/revisits_map.html")

In [ ]:
## Plotting FOVs

## Make a folium map
m = fov_df.explore(color="color", tooltip=["satellite", "time"])

## Add WRS2
# wrs2 = gpd.read_file('./WRS2_descending_0/WRS2_descending.shp')
# wrs2 = wrs2.cx[xmin: xmax, ymin: ymax]
# folium.GeoJson(data=wrs2["geometry"], overlay=False).add_to(m)

## View or save
m#.save("./tmp/fovs_map.html")